<a href="https://colab.research.google.com/github/pullz6/Research_inspired_Self-Explaining-Neural-Networks-for-Business-Process-Monitoring/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing libraries

In [3]:
!pip install mlflow -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 99.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 700.2/700.2 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.2 MB/s eta 0:00:00


In [4]:
!pip install pyngrok -q

In [7]:
#Importing libraries
import mlflow as ml
import subprocess
from pyngrok import ngrok, conf
import getpass
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import datetime
from sklearn.preprocessing import MinMaxScaler

In [8]:
df = pd.read_csv("Helpdesk.csv")

In [9]:
df.head()

,concept:name,lifecycle:transition,org:resource,time:timestamp,Activity,Resource,case:concept:name,case:variant,case:variant-index,case:creator
0,Assign seriousness,complete,Value 2,2010-01-13 08:40:25+02:00,Assign seriousness,Value 2,Case3608,Variant 33,33,Fluxicon Disco
1,Take in charge ticket,complete,Value 2,2010-01-29 08:52:27+02:00,Take in charge ticket,Value 2,Case3608,Variant 33,33,Fluxicon Disco
2,Resolve ticket,complete,Value 2,2010-01-29 08:52:34+02:00,Resolve ticket,Value 2,Case3608,Variant 33,33,Fluxicon Disco
3,Closed,complete,Value 5,2010-02-13 08:52:48+02:00,Closed,Value 5,Case3608,Variant 33,33,Fluxicon Disco
4,Closed,complete,Value 5,2010-02-13 08:52:48+02:00,Closed,Value 5,Case3608,Variant 33,33,Fluxicon Disco


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21348 entries, 0 to 21347
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   concept:name          21348 non-null  object
 1   lifecycle:transition  21348 non-null  object
 2   org:resource          21348 non-null  object
 3   time:timestamp        21348 non-null  object
 4   Activity              21348 non-null  object
 5   Resource              21348 non-null  object
 6   case:concept:name     21348 non-null  object
 7   case:variant          21348 non-null  object
 8   case:variant-index    21348 non-null  int64 
 9   case:creator          21348 non-null  object
dtypes: int64(1), object(9)
memory usage: 1.6+ MB


In [11]:
df.drop_duplicates(inplace=True)

In [12]:
df['case:variant-index'] = df['case:variant-index'].apply(lambda x: int(x))

In [13]:
df['Resource'] = df['Resource'].str.strip().str[-1]

In [14]:
df['Resource'] = df['Resource'].apply(lambda x: int(x))

In [15]:
df['time:timestamp'] = pd.to_datetime(df['time:timestamp'], utc=True)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21221 entries, 0 to 21347
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   concept:name          21221 non-null  object             
 1   lifecycle:transition  21221 non-null  object             
 2   org:resource          21221 non-null  object             
 3   time:timestamp        21221 non-null  datetime64[ns, UTC]
 4   Activity              21221 non-null  object             
 5   Resource              21221 non-null  int64              
 6   case:concept:name     21221 non-null  object             
 7   case:variant          21221 non-null  object             
 8   case:variant-index    21221 non-null  int64              
 9   case:creator          21221 non-null  object             
dtypes: datetime64[ns, UTC](1), int64(2), object(7)
memory usage: 1.8+ MB


In [17]:
df['new_date'] = df['time:timestamp'].dt.date
df['new_date'] = pd.to_datetime(df['new_date'], utc=True)
df['new_time'] = pd.to_datetime(
    df['time:timestamp'].dt.time.astype(str),  # Convert time to string
    format='%H:%M:%S'  # Parse as pure time
).dt.floor('S')

<ipython-input-17-16d55716170b>:6: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  ).dt.floor('S')


In [18]:
min_timestamps = df.groupby('case:concept:name')['time:timestamp'].transform('min')

In [19]:
df['time_lapsed'] = df['time:timestamp'] - min_timestamps

In [20]:
df['time_lapsed'] = pd.to_timedelta(df['time_lapsed'])

In [21]:
df['time_lapsed'] = df['time_lapsed'].dt.total_seconds()

In [22]:
df['concept:name'].unique()

array(['Assign seriousness', 'Take in charge ticket', 'Resolve ticket',
       'Closed', 'Wait', 'Create SW anomaly', 'Insert ticket',
       'Schedule intervention', 'RESOLVED', 'INVALID', 'VERIFIED',
       'Resolve SW anomaly', 'Require upgrade', 'DUPLICATE'], dtype=object)

In [23]:
#Checking if we can concept:name and activity is the same as well as the org:resource and the resource, so that we can delete this feild.
df['Checker_1'] = (df['concept:name'] == df['Activity'])
df['Checker_2'] = (df['org:resource'] == df['Resource'])

In [24]:
#Let's see if it all the same, if True.
df['Checker_2'].unique()

array([False])

In [25]:
#Lets drop it since it is the same
df.drop(['concept:name','org:resource','Checker_1','Checker_2','case:creator','lifecycle:transition','case:variant'],axis=1, inplace=True)

In [26]:
#Lets create a stage number for each activity stage of the help desk, this will help us easily identify where the case is in terms of the process

#This is also to immulate the label encoder
#First create a dataframe with stage numbers and the stage names
stage = pd.DataFrame()
stage['Stage_no'] = [1,2,3,4,5,6,7,8,9,9.1,9.2,10]
stage['Activity'] = ['Insert ticket','Assign seriousness','Take in charge ticket','Wait','Require upgrade','Schedule intervention','Resolve ticket','VERIFIED','RESOLVED','INVALID', 'DUPLICATE','Closed']

In [27]:
df=pd.merge(df,stage,on='Activity', how='left')

In [28]:
df.head()

,time:timestamp,Activity,Resource,case:concept:name,case:variant-index,new_date,new_time,time_lapsed,Stage_no
0,2010-01-13 06:40:25+00:00,Assign seriousness,2,Case3608,33,2010-01-13 00:00:00+00:00,1900-01-01 06:40:25,0.0,2.0
1,2010-01-29 06:52:27+00:00,Take in charge ticket,2,Case3608,33,2010-01-29 00:00:00+00:00,1900-01-01 06:52:27,1383122.0,3.0
2,2010-01-29 06:52:34+00:00,Resolve ticket,2,Case3608,33,2010-01-29 00:00:00+00:00,1900-01-01 06:52:34,1383129.0,7.0
3,2010-02-13 06:52:48+00:00,Closed,5,Case3608,33,2010-02-13 00:00:00+00:00,1900-01-01 06:52:48,2679143.0,10.0
4,2010-01-13 10:26:04+00:00,Assign seriousness,2,Case2748,1,2010-01-13 00:00:00+00:00,1900-01-01 10:26:04,0.0,2.0


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21221 entries, 0 to 21220
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   time:timestamp      21221 non-null  datetime64[ns, UTC]
 1   Activity            21221 non-null  object             
 2   Resource            21221 non-null  int64              
 3   case:concept:name   21221 non-null  object             
 4   case:variant-index  21221 non-null  int64              
 5   new_date            21221 non-null  datetime64[ns, UTC]
 6   new_time            21221 non-null  datetime64[ns]     
 7   time_lapsed         21221 non-null  float64            
 8   Stage_no            21142 non-null  float64            
dtypes: datetime64[ns, UTC](2), datetime64[ns](1), float64(2), int64(2), object(2)
memory usage: 1.5+ MB


In [30]:
df.head()

,time:timestamp,Activity,Resource,case:concept:name,case:variant-index,new_date,new_time,time_lapsed,Stage_no
0,2010-01-13 06:40:25+00:00,Assign seriousness,2,Case3608,33,2010-01-13 00:00:00+00:00,1900-01-01 06:40:25,0.0,2.0
1,2010-01-29 06:52:27+00:00,Take in charge ticket,2,Case3608,33,2010-01-29 00:00:00+00:00,1900-01-01 06:52:27,1383122.0,3.0
2,2010-01-29 06:52:34+00:00,Resolve ticket,2,Case3608,33,2010-01-29 00:00:00+00:00,1900-01-01 06:52:34,1383129.0,7.0
3,2010-02-13 06:52:48+00:00,Closed,5,Case3608,33,2010-02-13 00:00:00+00:00,1900-01-01 06:52:48,2679143.0,10.0
4,2010-01-13 10:26:04+00:00,Assign seriousness,2,Case2748,1,2010-01-13 00:00:00+00:00,1900-01-01 10:26:04,0.0,2.0


In [31]:
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [32]:
X_sequences = []
y_act = []
y_time = []

In [33]:
case_groups = df.groupby('case:concept:name')
max_len = 14  # As per your model's expected timesteps
num_features = 26  # Must match your feature count

In [34]:
import numpy as np
a = np.array((7,2,3))
b = np.array((2,9,4))
np.column_stack((a,b))

array([[7, 2],
       [2, 9],
       [3, 4]])

In [35]:
for _, group in case_groups:
    group = group.sort_values('time:timestamp')

    # Create feature matrix (ensure no NaT/NaN)
    try:
        features = np.column_stack([
            group['Stage_no'].values,
            group['Resource'].values,
            group['case:variant-index'].values,
            #group['time_lapsed'].astype('float32').values.reshape(-1, 1),
            #group['new_date'].dt.dayofweek.values / 6,
            #group['new_time'].dt.hour.values / 23
        ])
    except Exception as e:
        print(f"Error in group {_}: {e}")
        continue  # Skip problematic groups

    # Pad sequences (handle empty cases)
    if features.size == 0:
        padded = np.zeros((max_len, n_features), dtype='float32')
    else:
        padded = pad_sequences(
            [features],
            maxlen=max_len,
            padding='post',
            truncating='post',
            dtype='float32'
        )[0]

    X_sequences.append(padded)
    y_act.append((group['Stage_no']).values)
    y_time.append(group['time_lapsed'].values)

In [36]:
X_sequences = np.array(X_sequences)
y_act = np.array(y_act, )
y_time = np.array(y_time)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (4580,) + inhomogeneous part.

In [ ]:
X_sequences

Model building

In [ ]:
!pip install keras

In [38]:
with mlflow.start_run(run_name="Testing"):
  mlflow.log_metric("m1", 2.0)
  mlflow.log_param("p1", "mlflow-colab")

In [40]:
get_ipython().system_raw("mlflow ui --port 5000 &")

In [42]:
ngrok.kill()

In [45]:
NGROK_AUTH_TOKEN = "you_key"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

In [46]:
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

MLflow Tracking UI: https://6122-35-226-120-102.ngrok-free.app


In [ ]:
#Creating the model architecture

In [39]:
#Importing the required libriares since the LSTM model will be done with Pytorch
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras.layers import Dense
from keras.layers import LSTM, GRU, SimpleRNN
from keras.layers import Input
from keras.utils import get_file
from keras.optimizers import Nadam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers import BatchNormalization

In [ ]:
main_input = Input(shape=(max_len, num_features), name='main_input')

In [ ]:
main_input

<KerasTensor shape=(None, 14, 26), dtype=float32, sparse=False, name=main_input>

In [ ]:
#Create the training split

In [ ]:
l1 = LSTM(100, implementation=2, kernel_initializer='glorot_uniform', return_sequences=True, dropout=0.2)(main_input) # the shared layer
b1 = BatchNormalization()(l1)
l2_1 = LSTM(100, implementation=2, kernel_initializer='glorot_uniform', return_sequences=False, dropout=0.2)(b1) # the layer specialized in activity prediction
b2_1 = BatchNormalization()(l2_1)
l2_2 = LSTM(100, implementation=2, kernel_initializer='glorot_uniform', return_sequences=False, dropout=0.2)(b1) # the layer specialized in time prediction
b2_2 = BatchNormalization()(l2_2)
act_output = Dense(len(df['Stage_no'].unique()), activation='softmax', kernel_initializer='glorot_uniform', name='act_output')(b2_1)
time_output = Dense(1, kernel_initializer='glorot_uniform', name='time_output')(b2_2)

In [ ]:
model = Model(inputs=[main_input], outputs=[act_output, time_output])

opt = Nadam(learning_rate=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, weight_decay=0.004, clipvalue=3)

model.compile(loss={'act_output':'categorical_crossentropy', 'time_output':'mae'}, optimizer=opt)
early_stopping = EarlyStopping(monitor='val_loss', patience=42)
model_checkpoint = ModelCheckpoint('output_files/models/model_{epoch:02d}-{val_loss:.2f}.h5', monitor='val_loss', verbose=0, save_best_only=True,
                                   save_weights_only=False, mode='auto')
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)

Model Training

In [ ]:
model.fit(X_sequences, {'act_output':y_act, 'time_output':y_time}, validation_split=0.2, verbose=2, callbacks=[early_stopping, model_checkpoint, lr_reducer], batch_size=max_len, epochs=500)

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed